In [43]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

Initializing the constants and typecasting to float32 precision.

In [44]:
ip_folder = "./q2_input"
op_folder = "./q2_output"

tending_zero = np.float32(0.000000001)
G = np.float32(6.67 * 10**(5))
threshold = np.float32(0.1)
time_step = np.float32(0.0001)
INF = np.float32(100000000000000000.0)

This is the tensorflow computational graph.

In [45]:
masses = np.load(ip_folder + "/masses.npy")
positions = np.load(ip_folder + "/positions.npy")
velocity = np.load(ip_folder + "/velocities.npy")
# N = total number of particles
N = D.size / 2

# Typecasting every floating points to float32 precision
masses = masses.astype(np.float32)
positions = positions.astype(np.float32)
veclocity = velocity.astype(np.float32)

'''Custom Input'''
# D = np.array([[-2,0],
#               [0,2],
#               [2,0]], dtype=np.float32)

# # M = np.array([[1.0, 2.0, 3.0]]).transpose()
# M = tf.transpose(tf.constant([[4, 4, 4]], dtype=tf.float32))
# V = np.array([[0,0],
#               [0,0],
#               [0,0]], dtype=np.float32)

# Initial X and Y coordinates of each particles.
# Shape = (N, 1)
initX = np.reshape(D[:,0], [-1,1])
initY = np.reshape(D[:,1], [-1,1])

# Initial velocity along X and Y directions for each particles
# Shape = (3,1)
initVx = np.reshape(V[:,0], [-1,1])
initVy = np.reshape(V[:,1], [-1,1])

# Shape (N, 1)
posX = tf.placeholder(dtype=tf.float32)
posY = tf.placeholder(dtype=tf.float32)

# Shape (N, 1)
velX = tf.placeholder(dtype=tf.float32)
velY = tf.placeholder(dtype=tf.float32)

# posX, posY = initX, initY
# velX, velY = initVx, initVy

# distance vector between each pair of particles
# Shape = (N, N)
delX = tf.math.add(-posX, tf.transpose(posX))
delY = tf.math.add(-posY, tf.transpose(posY))

# Masses multiplied between each pair of particles
# Shape = (N, N)
mimj = tf.math.multiply(M, tf.transpose(M))

# Force matrix between each pair of particles
# Shape (N, N)
Fy = -(G * mimj * (-delY)) / (tf.math.pow(tf.math.sqrt(tf.math.pow(delX, 2) + tf.math.pow(delY, 2)), 3) + tending_zero)
Fx = -(G * mimj * (-delX)) / (tf.math.pow(tf.math.sqrt(tf.math.pow(delX, 2) + tf.math.pow(delY, 2)), 3) + tending_zero)

# Acceleration for each particles
# Shape = (N, 1)
ax = tf.reshape(tf.reduce_sum(Fx, axis=1), [-1, 1]) / M
ay = tf.reshape(tf.reduce_sum(Fy, axis=1), [-1, 1]) / M

# s1 = s0 + ut + 1/2 at^2
# Shape = (N, 1)
new_X = posX + velX * time_step + 0.5 * ax * time_step * time_step
new_Y = posY + velY * time_step + 0.5 * ay * time_step * time_step

# v1 = v0 + at
# Shape = (N, 1)
new_velX = velX + ax * time_step
new_velY = velY + ay * time_step

In [46]:
init_op = tf.global_variables_initializer()
# Create a summary to monitor positions
tf.summary.scalar("posX", new_X)
tf.summary.scalar("posY", new_Y)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

Running the session.

In [ ]:
with tf.Session() as sess:
#     Initializing the global variables
    sess.run(init_op)
#     Writing the summary log of the graph. Open it with tensorboard
    summary_writer = tf.summary.FileWriter("logs", graph=tf.get_default_graph())
    summary_writer.flush()
    summary_writer.close()
#     i = number of iterations
    i = 0
    while(True):
        i += 1
        [initX, initY, initVx, initVy] = sess.run([new_X, new_Y, new_velX, new_velY], feed_dict={posX: initX, posY: initY, velX: initVx, velY: initVy})
#         distance vector between each pair of particles
#         Shape = (N, N)
        delX = tf.math.add(-initX, tf.transpose(initX))
        delY = tf.math.add(-initY, tf.transpose(initY))
#         Actual distance between each pair of particles
        dist_all_pairs = tf.math.sqrt(tf.math.pow(delX, 2) + tf.math.pow(delY, 2))
#         Diagonal will be 0, setting it to INFINITE
        dist_all_pairs = tf.matrix_set_diag(dist_all_pairs, tf.fill(dist_all_pairs.shape[0:-1], INF))
        min_dist = tf.math.reduce_min(dist_all_pairs)
        print "Iteration: " + `i` + " Min dist: " + `sess.run(min_dist)`
        if(sess.run(min_dist) <= threshold):
            break
    final_position = np.concatenate((initX, initY), axis = 1)
    final_velocity = np.concatenate((initVx, initVy), axis = 1)
    print "Final Position: \n", final_position
    print "Final Velocity: \n", final_velocity
    np.save(op_folder + "/final_position.npy", final_position)
    np.save(op_folder + "/final_velocity.npy", final_velocity)

Iteration: 1 Min dist: 4.5487475
Iteration: 2 Min dist: 4.0849156
Iteration: 3 Min dist: 3.2594974
Iteration: 4 Min dist: 1.9196326
Iteration: 5 Min dist: 0.6405992
Iteration: 6 Min dist: 2.519534
Iteration: 7 Min dist: 0.47674438
Iteration: 8 Min dist: 4.5882845
Iteration: 9 Min dist: 4.572165
Iteration: 10 Min dist: 2.3751042
Iteration: 11 Min dist: 3.7533915
Iteration: 12 Min dist: 2.1574805
Iteration: 13 Min dist: 0.39782473
Iteration: 14 Min dist: 1.2374668
Iteration: 15 Min dist: 3.1594968
Iteration: 16 Min dist: 4.2667336
Iteration: 17 Min dist: 4.213464
Iteration: 18 Min dist: 4.159133
Iteration: 19 Min dist: 4.104541
Iteration: 20 Min dist: 4.050716
Iteration: 21 Min dist: 3.0452914
Iteration: 22 Min dist: 3.0014107
Iteration: 23 Min dist: 1.4574661
Iteration: 24 Min dist: 2.4176652
Iteration: 25 Min dist: 3.587228
Iteration: 26 Min dist: 3.6307712
Iteration: 27 Min dist: 1.1399543
Iteration: 28 Min dist: 3.9208777
Iteration: 29 Min dist: 2.4853425
Iteration: 30 Min dist: 0.90

Iteration: 239 Min dist: 3.9719043
Iteration: 240 Min dist: 3.8651552
Iteration: 241 Min dist: 3.7568393
Iteration: 242 Min dist: 3.6468766
Iteration: 243 Min dist: 3.535176
Iteration: 244 Min dist: 3.4216366
Iteration: 245 Min dist: 3.3061492
Iteration: 246 Min dist: 3.1885931
Iteration: 247 Min dist: 3.0688353
Iteration: 248 Min dist: 2.946721
Iteration: 249 Min dist: 2.8220842
Iteration: 250 Min dist: 2.6947265
Iteration: 251 Min dist: 2.56443
Iteration: 252 Min dist: 2.4309385
Iteration: 253 Min dist: 1.6033764
Iteration: 254 Min dist: 0.6886963
Iteration: 255 Min dist: 2.0081904
Iteration: 256 Min dist: 1.858525
Iteration: 257 Min dist: 0.8861335
Iteration: 258 Min dist: 1.5429262
Iteration: 259 Min dist: 1.3756329
Iteration: 260 Min dist: 1.2010013
